s = [] very good
split


# data

In [1]:
from os import listdir
from os.path import isfile, join
easy_ham = [f for f in listdir("data/easy_ham") if isfile(join("data/easy_ham", f))] # 2500 = 1716+784
easy_ham_2 = [f for f in listdir("data/easy_ham_2") if isfile(join("data/easy_ham_2", f))] # 1400 = 960+440
hard_ham = [f for f in listdir("data/hard_ham") if isfile(join("data/hard_ham", f))] # 250 = 172+78
spam = [f for f in listdir("data/spam") if isfile(join("data/spam", f))] # 500 = 344+156
spam_2 = [f for f in listdir("data/spam_2") if isfile(join("data/spam_2", f))] # 1396 = 958+438
for index in range(len(easy_ham)):
    easy_ham[index] = "data/easy_ham/"+ easy_ham[index]
for index in range(len(easy_ham_2)):
    easy_ham_2[index] = "data/easy_ham_2/"+ easy_ham_2[index]
for index in range(len(hard_ham)):
    hard_ham[index] = "data/hard_ham/"+ hard_ham[index]
for index in range(len(spam)):
    spam[index] = "data/spam/"+ spam[index]
for index in range(len(spam_2)):
    spam_2[index] = "data/spam_2/"+ spam_2[index]

In [2]:
ham = easy_ham + easy_ham_2 + hard_ham
spam = spam + spam_2

In [3]:
from sklearn.model_selection import train_test_split
Training_Ham, Testing_Ham = train_test_split(ham, test_size = 0.1, random_state = 1)
Training_Spam, Testing_Spam = train_test_split(spam, test_size = 0.1, random_state = 1)

In [4]:
print(len(Training_Ham), len(Testing_Ham), len(Training_Spam), len(Testing_Spam))

3735 415 1706 190


In [5]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
s=set(stopwords.words('english'))
# s = []

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
import re
import codecs
frequency_list = [1, 2, 3, 5, 10, 20, 30, 50, 100, 200, 300]
# frequency_list = [3, 50, 1000]
dictionary = dict()
Training = Training_Ham + Training_Spam
Testing = Testing_Ham + Testing_Spam

In [7]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
f = open("ret/KNN.txt", "a")
for frequency in frequency_list:
    print('frequency is ' + str(frequency))
    f.write('\nfrequency is ' + str(frequency) + '\n')
    dictionary = dict()

    for file_names in Training:
        #f = open(file_names, "r")
        ## UnicodeDecodeError: 'cp950' codec can't decode byte 0x93 in position 1885: illegal multibyte sequence
        #f = open(file_names, "r", encoding="utf-8")
        ## UnicodeDecodeError: 'utf-8' codec can't decode byte 0x93 in position 1885: invalid start byte

        file = codecs.open(file_names, 'r', encoding='utf-8', errors='ignore')

        body = file.read().split("\n\n", 1)[1].lower()
        word = re.findall(r"[a-zA-Z']+", body) # 123656 ## 21224
        word = body.split(' ')
        for i in word:
            dictionary[i] = dictionary.get(i, 0) + 1
#     print(len(dictionary))
    dictionary = {i:dictionary[i] for i in dictionary if dictionary[i]>frequency}
#     print(len(dictionary))
    #print(dictionary)
    list_dictionary = list(dictionary)
    list_dictionary = list(filter(lambda w: not w in s,list_dictionary))
    list_dictionary
#     print(len(list_dictionary))

    # Remove common word in dictionary
    for s__ in s:
        if s__ in dictionary:
            del dictionary[s__]
    print('dict length: ' + str(len(dictionary)))
    f.write('dict length: ' + str(len(dictionary)))
    count = 0
    for i in dictionary:
    #     print(i)
        dictionary[i] = count
        count += 1
    dictionary

    train_ndarray = np.zeros((len(Training), len(dictionary)), dtype = int)
    train_ndarray.shape

    label_train = [0] * len(Training_Ham) + [1] * len(Training_Spam)
    label_test = [0] * len(Testing_Ham) + [1] * len(Testing_Spam)

    count = 0
    for Ham_mail in Training_Ham:
        file = codecs.open(Ham_mail, 'r', encoding='utf-8', errors='ignore')
        body = file.read().split("\n\n", 1)[1].lower()
        word = re.findall(r"[a-zA-Z']+", body)
        word = body.split(' ')
        for w in word:
            if w in dictionary:
                train_ndarray[count][dictionary[w]] = 1
        count += 1
    for Spam_mail in Training_Spam:
        file = codecs.open(Spam_mail, 'r', encoding='utf-8', errors='ignore')
        body = file.read().split("\n\n", 1)[1].lower()
        word = re.findall(r"[a-zA-Z']+", body)
        word = body.split(' ')
        for w in word:
            if w in dictionary:
                train_ndarray[count][dictionary[w]] = 1
        count += 1
    train_dataframe = pd.DataFrame(train_ndarray)

    test_ndarray = np.zeros((len(Testing), len(dictionary)), dtype = int)
    test_ndarray.shape

    count = 0
    for Ham_mail in Testing_Ham:
        file = codecs.open(Ham_mail, 'r', encoding='utf-8', errors='ignore')
        body = file.read().split("\n\n", 1)[1].lower()
        word = re.findall(r"[a-zA-Z']+", body)
        word = body.split(' ')
        for w in word:
            if w in dictionary:
                test_ndarray[count][dictionary[w]] = 1
    #     print(train_ndarray[count])
        count += 1
    for Spam_mail in Testing_Spam:
        file = codecs.open(Spam_mail, 'r', encoding='utf-8', errors='ignore')
        body = file.read().split("\n\n", 1)[1].lower()
        word = re.findall(r"[a-zA-Z']+", body)
        word = body.split(' ')
        for w in word:
            if w in dictionary:
                test_ndarray[count][dictionary[w]] = 1
        count += 1
    test_dataframe = pd.DataFrame(test_ndarray)

    neigh = KNeighborsClassifier(n_neighbors = 3)
    neigh.fit(train_dataframe, label_train)

    result = neigh.predict(test_dataframe)


    #-------------- from block -----------------

#     Spam_from = dict()
#     count = 0
#     for Spam_mail in Training_Spam:
#         file = codecs.open(Spam_mail, 'r', encoding='utf-8', errors='ignore')
#         word = re.findall(r"From.*\n", file.read())[0]
#         word_2 = re.findall(r"[\w*\.?]*@[\w*\.?]*", word)
#         if not word_2 :
#             continue
#         Spam_from[word_2[0]] = Spam_from.get(i, 0)    

#     index = 0
#     cheat_result = result.copy()
#     for mail in Testing:
#         file = codecs.open(mail, 'r', encoding='utf-8', errors='ignore')
#         word = re.findall(r"From.*\n", file.read())[0]
#         word_2 = re.findall(r"[\w*\.?]*@[\w*\.?]*", word)
#         if word_2[0] in Spam_from:
#             cheat_result[index] = 1
#         index += 1

#     eva_fun_2(label_test, cheat_result, 'NN')

    #--------------- end ---------------------

#     print('neighbor is :' + str(nn) + '\n')
#     f.write('neighbor is :' + str(nn) + '\n')

    tn, fp, fn, tp = confusion_matrix(label_test, result).ravel()

    Accuracy = (tp+tn)/(tn+ fp+ fn+ tp)
    Precision = (tp)/(fp+ tp)
    Recall = (tp)/(fn+ tp)
    print('Recall\t\tPrecision\tAccuracy')
    f.write('\nRecall\t\tPrecision\tAccuracy\n')
    print(str(round(Recall*100, 2))+'\t\t'+ str(round(Precision*100, 2))+'\t\t'+str(round(Accuracy*100, 2)))
    f.write(str(round(Recall*100, 2))+'\t\t'+ str(round(Precision*100, 2))+'\t\t'+str(round(Accuracy*100, 2)))
    print('\n------------------------------------\n')
    f.write('\n------------------------------------\n')
f.close()

frequency is 1
dict length: 83608
Recall		Precision	Accuracy
93.68		63.12		80.83

------------------------------------

frequency is 2
dict length: 48665
Recall		Precision	Accuracy
94.74		63.16		80.99

------------------------------------

frequency is 3
dict length: 36475
Recall		Precision	Accuracy
94.21		62.37		80.33

------------------------------------

frequency is 5
dict length: 24595
Recall		Precision	Accuracy
94.21		63.03		80.83

------------------------------------

frequency is 10
dict length: 12954
Recall		Precision	Accuracy
93.16		65.8		82.64

------------------------------------

frequency is 20
dict length: 7319
Recall		Precision	Accuracy
83.68		73.95		85.62

------------------------------------

frequency is 30
dict length: 5031
Recall		Precision	Accuracy
82.63		71.36		84.13

------------------------------------

frequency is 50
dict length: 3140
Recall		Precision	Accuracy
78.95		72.46		83.97

------------------------------------

frequency is 100
dict length: 1560
Recal

In [8]:
len(list_dictionary)

429

In [9]:
# count = 0
# cheat_result = result.copy()
# for mail in Testing:
#     file = codecs.open(mail, 'r', encoding='utf-8', errors='ignore')
# #     print(file.read())
#     word = re.findall(r"From.*\n", file.read())[0]
# #     print(Spam_mail)
#     word_2 = re.findall(r"[\w*\.?]*@[\w*\.?]*", word)
# #     if word_2[0] == "admin@xent.com":
# #         continue
# #     if word_2[0] == "admin@lists.sourceforge.net":
# #         continue
# #     if word_2[0] == "admin@linux.ie":
# #         continue
#     if word_2[0] in Spam_from:
#         cheat_result[count] = 1
#         print(word_2[0])
# #     print(word_2[0])
#     count += 1